In [1]:
import torch
import os
import datetime

%matplotlib inline

from selene_sdk.utils import load_path
from selene_sdk.utils import parse_configs_and_run
from selene_sdk.utils import DeeperDeepSEA
from selene_sdk.predict._common import predict

In [2]:
import pandas as pd

In [3]:
import numpy as np

In [4]:
from selene_sdk.utils import NonStrandSpecific


model_architecture = NonStrandSpecific(DeeperDeepSEA(2000, 1846))


In [6]:
from selene_sdk.predict import AnalyzeSequences
from selene_sdk.utils import load_features_list


features = load_features_list("/rds-d5/project/who1000/rds-who1000-wgs10k/user/tt419/Selene_data/selene_ftp_output/distinct_features.txt")
analysis = AnalyzeSequences(
    model_architecture,
    "/rds-d5/project/who1000/rds-who1000-wgs10k/user/tt419/Selene_data/selene_ftp_output/logs/online_sampler_outputs_base/best_model.pth.tar",
    sequence_length=2000,
    features=features,
    use_cuda=False)

#analysis.in_silico_mutagenesis_from_file("sequences.fasta", 
#                                         save_data=["abs_diffs", "logits", "predictions"], 
#                                         output_dir=".",
#                                         use_sequence_name=False)

In [7]:
import requests

server = "https://grch37.rest.ensembl.org"

def get_reference(chrom, start, end=None, species="human", GRCh="GRCh37"):
    if not end:
        end = start+5000
    assert start <= end, "Start has to be before end"
    assert end - start < 10000000, "Can't fetch more then 10 Mb (10 million bases)"
    sub_path = f"/sequence/region/{species}/"
    region = f"{chrom}:{start}..{end}"
    r = requests.get(server + sub_path + region, headers={"Content-Type": "text/plain", "coord_system_version": str(GRCh)})
    if r.ok:
        return r.text
    else:
        r.raise_for_status()
        return -1

def find_variant_in_reference(variant_coord, reference, ref_start, length=2000, focus=200):
    """
    @param variant_coord: (chrom, pos,ref, alt)
    """
    index = int(variant_coord[1])-int(ref_start)
    ref_length = len(variant_coord[2])
    ref_ens = reference[index:(index+ref_length)]
    # if ref_ens == variant_coord[2]:
    #     print(f"The reference is not the same!Ensemble: {ref_ens}, variant_ref: {variant_coord[2]}")

    #assert ref_ens == variant_coord[2], f"The reference is not the same!Ensemble: {reference[index-1:(index+ref_length)]}, variant_ref: {variant_coord[2]}"
    assert len(variant_coord[3]) <= focus, f"The alternate length ({len(variant_coord[3])}) is longer than the focus of the network: {focus} "
    result = []
    start_index = int(index-(length-focus)/2-(focus-len(variant_coord[3]))/2)
    end_index = int(index + ref_length + (length-focus)/2 + (focus-len(variant_coord[3]))/2)
    # The following is handling all cases: Substitution, Insertion and Deletion. It takes the whole reference up to the
    #   variant start, then appends the variant itself (in case of a deletion it is short than before, in case of an insertion longer)
    #   then adds the last bit of the reference, from starting index+ref_length to our result. This way the length difference of
    #   InDels is handled.
    result.extend(reference[start_index:index])
    result.extend(variant_coord[3])
    result.extend(reference[index + ref_length:end_index])
    result = "".join(result)
    diff = length-len(result)
    if diff > 0:
        print(f"Adding {diff} basepairs in the end.")
        result.append(reference[end_index:end_index+diff])
    elif diff < 0:
        print(f"Erasing {diff} basepairs in the end")
        result = result[:length]
    return result, reference[start_index:start_index+length]

def one_hot(sequence):  
    classes = 4    
    encode = {"A":0, "C":1, "G":2, "T":3}
    num_sequence = list(map(encode.get, sequence))    
    targets = np.array([num_sequence]).reshape(-1)    
    result = np.eye(classes)[np.array(targets).reshape(-1)]      
    return result.reshape(list(targets.shape)+[classes])    

In [14]:
def iterate_through_mutations_in_sequence(ref_seq, ref_start, var_df):
    for index, var in var_df.iterrows():
        yield find_variant_in_reference((var.chromosome, var.start, var.reference, var.alternate), ref_seq, ref_start)

def get_ref_for_gene(var_df, frame_length=1000): 
    chrom = var_df["chromosome"][0]
    start = var_df["start"].min() - frame_length
    end = var_df["end"].max() + frame_length
    return (chrom, start, end),get_reference(chrom, start, end)           
        
        
def load_gene(fp):
    var_df = pd.read_pickle(fp)
    return var_df[var_df.columns[:5].tolist()+var_df.columns[-6:].tolist()+
                  var_df.columns[5:-6].tolist()]



def predict_whole_gene(analysis, var_df):
    coord, reference = get_ref_for_gene(var_df)
    if coord[1] <= 55267612 and coord[2] >= 55267612:
        print("Found rs13266183")#
    if coord[1] <= 55258127 and coord[2] >= 55258127:
        print("Found rs10103692")

    iter_count = 0
    if reference == -1:
        return np.array([-1])
    for frame,ref_frame in iterate_through_mutations_in_sequence(reference, coord[1], var_df):
        encoded_frame = one_hot(frame)
        encoded_ref_frame = one_hot(ref_frame)
        sequences = np.stack([encoded_ref_frame.astype(int), encoded_frame.astype(int)], 0)
        yield predict(analysis.model, sequences, use_cuda=analysis.use_cuda) 
        iter_count += 1
        if iter_count % 100 == 0:
            print(f"Iteration {iter_count} done.")
    print(f"Done in {iter_count} steps.")        

In [15]:
def run(analysis, gene_dir=".", new_fp_dir=""):
    
    print(f"It is now {datetime.datetime.now()}")
    # iterate through all genes in gene_dir, grab reference, predict each variant, save in separate directory -> save in pandas df
    print(f"Running with gene_dir = {gene_dir}, and new_fp_dir = {new_fp_dir}")
    for root, dirs, files in os.walk(gene_dir):
        for filename in files:
            print(f"There is a file {filename}")
            if ".pickle" in filename[-8:] and "avro37_all" in filename[:11] and "SOX17" in filename:
                try:
                    print(f"Loading {filename}")
                    gene = load_gene(os.path.join(gene_dir,filename))
                    gene_name = "_".join(filename.split("_")[2:])[:-7]
                    new_fp = f"{new_fp_dir}predicted_{gene_name}.pickle"
                    if os.path.isfile(new_fp):
                        print(f"Gene {gene_name} is already in {new_fp}")
                        continue
                    print(f"Predicting for {gene_name}")
                    gene["prediction"] = list(predict_whole_gene(analysis, gene))
                    gene.to_pickle(new_fp)
                    print(f"Done with {gene_name}")
                    print(f"It is now {datetime.datetime.now()}")
                except Exception as e:
                    print(f"Something went wrong in {filename}")
                    
                    raise

def main():
    gene_dir = "/home/tt419/Projects/data/104genes_37_acan/"
    # Run with my model
    run(analysis ,gene_dir=gene_dir, new_fp_dir="../logs/myImpl_37_98genes-gnomad/")
    print("Done with first predictions")
    # Run with pretrained model by zhang (troyanskaya group)
    #run(gene_dir=gene_dir,model_fp="../deepsea_keras.h5", \
    #new_fp_dir="../logs/zhangImpl_AGCT/")

In [16]:
gene_dir = "/home/tt419/Projects/data/104genes_37_acan/"
# Run with my model
run(analysis ,gene_dir=gene_dir, new_fp_dir="/rds-d5/project/who1000/rds-who1000-wgs10k/user/tt419/Selene_data/selene_ftp_output/logs/mySelene_37_104genes-gnomad/")


# rs13266183 = [8, 55267612  , 55267612]
# rs10103692 = [8, 55258127  , 55258127]

# start = var_df["start"].min() - frame_length
# end = var_df["end"].max() + frame_length
# (chrom, start, end),get_reference(chrom, start, end)  


# new_fp_dir="/rds-d5/project/who1000/rds-who1000-wgs10k/user/tt419/Selene_data/selene_ftp_output/logs/mySelene_37_104genes-gnomad/"
# print(f"Running with gene_dir = {gene_dir}, and new_fp_dir = {new_fp_dir}")
# for root, dirs, files in os.walk(gene_dir):
#     for filename in files:
#         print(f"There is a file {filename}")
#         if ".pickle" in filename[-8:] and "avro37_all" in filename[:11]:
#             try:
#                 print(f"Loading {filename}")
#                 gene = load_gene(os.path.join(gene_dir,filename))
#                 gene_name = "_".join(filename.split("_")[2:])[:-7]
#                 new_fp = f"{new_fp_dir}predicted_{gene_name}.pickle"
#                 if os.path.isfile(new_fp):
#                     print(f"Gene {gene_name} is already in {new_fp}")
#                     continue
#                 print(f"Predicting for {gene_name}")
#                 var_df = gene
#                 coord, reference = get_ref_for_gene(var_df)
#                 iter_count = 0
#                 if reference == -1:
#                     np.array([-1])
#                 for frame,ref_frame in iterate_through_mutations_in_sequence(reference, coord[1], var_df):
#                     encoded_frame = one_hot(frame)
#                     encoded_ref_frame = one_hot(ref_frame)
#                     sequences = np.stack([encoded_ref_frame.astype(int), encoded_frame.astype(int)], 0)
#                     tmp = predict(analysis.model, sequences, use_cuda=analysis.use_cuda) 
#                     break
#             except Exception as e:
#                     print(f"Something went wrong in {filename}")
                    
#                     raise
#         break
#     break

It is now 2021-04-26 17:47:20.508754
Running with gene_dir = /home/tt419/Projects/data/104genes_37_acan/, and new_fp_dir = /rds-d5/project/who1000/rds-who1000-wgs10k/user/tt419/Selene_data/selene_ftp_output/logs/mySelene_37_104genes-gnomad/
There is a file avro37_allACAN_HIF1A.pickle
There is a file avro37_allACAN_SIRT3.pickle
There is a file avro37_allACAN_RNF213.pickle
There is a file avro37_allACAN_KLF2.pickle
There is a file avro37_allACAN_TBX4.pickle
There is a file avro37_allACAN_SBF1.pickle
There is a file avro37_allACAN_BMPR1A.pickle
There is a file avro37_allACAN_TNF.pickle
There is a file avro37_allACAN_SDHC.pickle
There is a file avro37_allACAN_CTSL.pickle
There is a file avro37_allACAN_PDGFD.pickle
There is a file avro37_allACAN_SMAD1.pickle
There is a file avro37_allACAN_EIF2AK4.pickle
There is a file avro37_allACAN_SOX4.pickle
There is a file avro37_allACAN_SOX17_PROMOTER_GENECARDS_2.pickle
Loading avro37_allACAN_SOX17_PROMOTER_GENECARDS_2.pickle
Gene SOX17_PROMOTER_GENEC